<a href="https://colab.research.google.com/github/OVP2023/CV/blob/main/%D0%BE%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%BB%D0%B8%D1%86_deepface_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install deepface

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 5.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.0/85.0 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 47.5 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=318cccc575ace8cb8851e2f385a6e9aefee8bc01b291eafaaceda04072f12707
  Stored in directory: /root/.cache/pip/wheels/46/54/24/1624fd5b8674eb1188623f7e8e17cdf7c0f6c24b609dfb8a89
Successfully built fire


In [ ]:
from deepface import DeepFace

25-02-19 12:09:46 - Directory /root/.deepface has been created
25-02-19 12:09:46 - Directory /root/.deepface/weights has been created


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#анализ лица на изображении
result = DeepFace.analyze(img_path="/content/drive/MyDrive/Datasets/CV/face/1.jpeg", actions=['age','gender','race','emotion'])

Action: emotion: 100%|██████████| 4/4 [00:01<00:00,  2.30it/s]


In [ ]:
result

[{'age': 37,
  'region': {'x': 264,
   'y': 179,
   'w': 369,
   'h': 369,
   'left_eye': (517, 321),
   'right_eye': (370, 318)},
  'face_confidence': 0.93,
  'gender': {'Woman': 0.07605688297189772, 'Man': 99.9239444732666},
  'dominant_gender': 'Man',
  'race': {'asian': 0.10631355689838529,
   'indian': 1.1669800616800785,
   'black': 0.09821857092902064,
   'white': 61.483317613601685,
   'middle eastern': 23.760822415351868,
   'latino hispanic': 13.384346663951874},
  'dominant_race': 'white',
  'emotion': {'angry': 2.763128899783925e-06,
   'disgust': 7.044992396642946e-13,
   'fear': 2.1645428032357624e-07,
   'happy': 99.20824756720633,
   'sad': 4.3402618588395875e-05,
   'surprise': 0.006394860282113884,
   'neutral': 0.7853171793004489},
  'dominant_emotion': 'happy'}]

In [ ]:
#Сравнение 2 лиц
verification = DeepFace.verify(img1_path="/content/drive/MyDrive/Datasets/CV/face/1.jpeg",img2_path="/content/drive/MyDrive/Datasets/CV/face/2.jpeg")

In [ ]:
verification

{'verified': False,
 'distance': 0.9391841088226728,
 'threshold': 0.68,
 'model': 'VGG-Face',
 'detector_backend': 'opencv',
 'similarity_metric': 'cosine',
 'facial_areas': {'img1': {'x': 264,
   'y': 179,
   'w': 369,
   'h': 369,
   'left_eye': (517, 321),
   'right_eye': (370, 318)},
  'img2': {'x': 397,
   'y': 257,
   'w': 168,
   'h': 168,
   'left_eye': (506, 322),
   'right_eye': (450, 324)}},
 'time': 3.68}

In [ ]:
#II построение своей сверточной модели
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

model = Sequential([
    Conv2D(32,(3,3), action = 'relu', input_shape=(224,224,3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(32,(3,3), action = 'relu', input_shape=(224,224,3)),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(512,activation='relu'),
    Dropout(0.5),
    Dense(128,activation='relu'), #вектор признаков лиц
    Dense(num_classes, activation='softmax') #классификация
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.fit(train_images, train_labels, epochs=10, batch_size=32, validation_data=(val_images, val_labels))

In [ ]:
predictions = model.predict(new_images)

In [ ]:
#III  Использование предобученных моделей для создания эмбединга изображения
from deepface.basemodels import Facenet

model = Facenet.loadModel()
embeddings = model.predict(preprocessed_image)